In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
from uuid import uuid4

In [3]:
from api import giobhutils as gio
from api.utils import parse_offset, today
from api.gui.radar2 import EditableGrid
from api.gui.collectors import DataCollector
from api.gui.params import curves
from api.ezutils import SQL

In [4]:
def get_var_query( 
    asof=None,
    book='MM',
    offset='2y'
):
    dt_fmt = '%Y%m%d'
    asof = pd.to_datetime(asof) if asof else today() - BDay()
    query = f"SELECT " \
        f"p.fund, " \
        f"p.book, " \
        f"p.strategy, " \
        f"p.minnb, " \
        f"p.positionName, " \
        f"p.riskCategory, " \
        f"p.currency, " \
        f"p.instrument, " \
        f"p.underlying_name, " \
        f"p.bhType, " \
        f"p.isAO, " \
        f"s.returnDate, " \
        f"s.returnValue " \
        f"FROM RiskManagement..vw_pos_openPositionMarketData p " \
        f"JOIN RiskVaR..vw_SeriesData s ON p.cobDate = s.cobDate and p.minbhid = s.minbhid " \
        f"WHERE 1=1 " \
        f"AND p.cobDate = '{asof:{dt_fmt}}' " \
        f"AND p.fund = 'BAL' " \
        f"AND p.book = '{book}' " \
        f"AND s.riskType = 'Total' " \
        f"AND s.returnHorizon = '1D' " \
        f"AND s.analysisHorizon = '1D' " \
        f"AND s.decayFactor = 1 " \
        f"AND s.exclude = 0 "
    
    if offset:
        offset = asof - parse_offset(offset)
        query = f"{query} AND s.returnDate >= {offset:{dt_fmt}}"
    return query

In [5]:
class VaRPnl:
    def get_data(
        self,
        asof=None, 
        book='MM', 
        offset='2y',
        exclude_cash=False,
        groupby='name',
    ):  
        res = gio.bhDbReadSql(
            '1', 
            sqlserver=SQL.RiskVar.server,
            sqldatabase=SQL.RiskVar.db,
            sqlquery=get_var_query(asof=asof, book=book, offset=offset)
        )

        self.res = res
        self.parse_res(res, exclude_cash, groupby)

    def parse_res(
        self,     
        res=None,
        exclude_cash=False,
        groupby='name',
    ):
        res = res if res is not None else self.res
        df = res.py
        df['returnDate'] = pd.to_datetime(df['returnDate'])
        ref_cols = [ 
            'fund', 
            'book', 
            'strategy', 
            'minnb', 
            'positionName', 
            'riskCategory',
            'currency', 
            'instrument', 
            'underlying_name',
            'isAO'
        ]

        
        if exclude_cash:
            df = df.loc[~df['positionName'].str.contains('CASH')]

        ref_df = df[ref_cols].drop_duplicates()
        if groupby:
            col = 'positionName' if groupby == 'name' else groupby
            hist_df = df.groupby([col, 'returnDate'], as_index=False, sort=False)['returnValue'].sum().pivot(index='returnDate', columns=col, values='returnValue')
            ref_df = ref_df.drop_duplicates(subset='positionName')
        else: 
            hist_df = df[['returnDate', 'minnb', 'returnValue']].pivot(index='returnDate', columns='minnb', values='returnValue')

        self.ref = ref_df
        self.hist = hist_df
        
    def var(self, df=None, days=521, q=0.05):
        df = df if df is not None else self.hist
        return df[:days].sum(axis=1).quantile(q)

In [6]:
def get_millify(decimal=2):
    def millify(value):
        n = abs(float(value))

        if n < 1000:
            return format(value, ".{}f".format(decimal))
        elif n < 1000000:
            return format(value / 1000, ".{}f".format(decimal)) + 'k'
        elif n < 1000000000:
            return format(value / 1000000, ".{}f".format(decimal)) + 'm'
        elif n < 1000000000000:
            return format(value / 1000000000, ".{}f".format(decimal)) + 'b'
        else:
            return format(value / 1000000000000, ".{}f".format(decimal)) + 't'

    return millify

In [7]:
v = VaRPnl()
v.get_data()
total_var = v.var()

In [8]:
total_var

-353030.39169999975

In [24]:
exclude_strategies = []  # ['MM:FX OVERLAY']

In [25]:
subset = v.ref.query(f"strategy not in {exclude_strategies}")
new_vars = {}
for pos in subset['positionName']:
    new_vars[pos] = v.var(v.hist.drop(pos, axis=1))
new_vars = pd.Series(new_vars).sort_values().rename('var')
new_vars = new_vars.to_frame()\
    .merge(v.ref[['positionName', 'strategy']]\
    .set_index('positionName'), how='left', left_index=True, right_index=True)\
    .set_index('strategy', append=True)\
    .reorder_levels([1, 0]).squeeze()
var_diff = (new_vars - total_var).sort_values(ascending=False).rename('diff')

In [26]:
var_diff.to_frame().style.bar(align='mid', color=['#d65f5f', '#5fba7d']).format(get_millify())

In [27]:
v.parse_res(groupby='strategy')
subset = v.ref.query(f"strategy not in {exclude_strategies}")
new_vars = {}
for strat in subset['strategy']:
    new_vars[strat] = v.var(v.hist.drop(strat, axis=1))
new_vars = pd.Series(new_vars).sort_values().rename('var')
var_diff = (new_vars - total_var).sort_values(ascending=False).rename('diff')

In [28]:
var_diff.to_frame().style.bar(align='mid', color=['#d65f5f', '#5fba7d']).format(get_millify())

In [ ]:
v.hist.drop('MM:FX OVERLAY', axis=1).sum(axis=1).quantile(0.05)

In [ ]:
(1180 - 148.45) / 1000

In [ ]:
v.hist.plot(backend='plotly', height=700)

In [ ]:
v.hist.drop('USD CME 3M SEP24', axis=1).sum(axis=1).quantile(0.05)

In [ ]:
drop = 'USD CME 3M SEP24'
ptf = v.hist.drop(drop, axis=1).sum(axis=1)

In [ ]:
pd.concat([ptf, v.hist[drop]], axis=1).corr()

In [10]:
v.parse_res(groupby='strategy')

In [18]:
v.hist.drop(['MM:USD HEDGE'], axis=1).sum(axis=1).quantile(0.05)

-584987.8671599999

In [14]:
total_var

-996441.6793000001